<a href="https://colab.research.google.com/github/wiv33/A-Learning-python/blob/master/machine-learning/_000_hello_machine/_000_basic/_003_cuk_edu/_004_3_text_processing/_002_ko_embedding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 케라스 임베딩 층을 이용한 워드 임베딩
출처 https://wikidocs.net/33793
케라스는 훈련 데이터의 단어들에 대해 워드 임베딩을 수행하는 도구 Embedding()을 제공합니다. Embedding()은 인공 신경망 구조 관점에서 임베딩 층(embedding layer)을 구현합니다.

1) 임베딩 층은 룩업 테이블이다.
임베딩 층의 입력으로 사용하기 위해서 입력 시퀀스의 각 단어들은 모두 정수 인코딩이 되어있어야 합니다.

어떤 단어 → 단어에 부여된 고유한 정수값 → 임베딩 층 통과 → 밀집 벡터

임베딩 층은 입력 정수에 대해 밀집 벡터(dense vector)로 맵핑하고 이 밀집 벡터는 인공 신경망의 학습 과정에서 가중치가 학습되는 것과 같은 방식으로 훈련됩니다. 훈련 과정에서 단어는 모델이 풀고자하는 작업에 맞는 값으로 업데이트 됩니다. 그리고 이 밀집 벡터를 임베딩 벡터라고 부릅니다.

정수를 밀집 벡터 또는 임베딩 벡터로 맵핑한다는 것은 어떤 의미일까요? 특정 단어와 맵핑되는 정수를 인덱스로 가지는 테이블로부터 임베딩 벡터 값을 가져오는 룩업 테이블이라고 볼 수 있습니다. 그리고 이 테이블은 단어 집합의 크기만큼의 행을 가지므로 모든 단어는 고유한 임베딩 벡터를 가집니다.



위의 그림은 단어 great이 정수 인코딩 된 후 테이블로부터 해당 인덱스에 위치한 임베딩 벡터를 꺼내오는 모습을 보여줍니다. 위의 그림에서는 임베딩 벡터의 차원이 4로 설정되어져 있습니다. 그리고 단어 great은 정수 인코딩 과정에서 1,918의 정수로 인코딩이 되었고 그에 따라 단어 집합의 크기만큼의 행을 가지는 테이블에서 인덱스 1,918번에 위치한 행을 단어 great의 임베딩 벡터로 사용합니다. 이 임베딩 벡터는 모델의 입력이 되고, 역전파 과정에서 단어 great의 임베딩 벡터값이 학습됩니다.

룩업 테이블의 개념을 이론적으로 우선 접하고, 처음 케라스를 배울 때 어떤 분들은 임베딩 층의 입력이 원-핫 벡터가 아니어도 동작한다는 점에 헷갈려 합니다. 케라스는 단어를 정수 인덱스로 바꾸고 원-핫 벡터로 한번 더 바꾸고나서 임베딩 층의 입력으로 사용하는 것이 아니라, 단어를 정수 인덱스로만 바꾼채로 임베딩 층의 입력으로 사용해도 룩업 테이블 된 결과인 임베딩 벡터를 리턴합니다.

케라스의 임베딩 층 구현 코드를 봅시다.

# 아래의 각 인자는 저자가 임의로 선정한 숫자들이며 의미있는 선정 기준이 아님.
v = Embedding(20000, 128, input_length=500)
# vocab_size = 20000
# output_dim = 128
# input_length = 500
임베딩 층은 다음과 같은 세 개의 인자를 받습니다.

vocab_size : 텍스트 데이터의 전체 단어 집합의 크기입니다.
output_dim : 워드 임베딩 후의 임베딩 벡터의 차원입니다.
input_length : 입력 시퀀스의 길이입니다. 만약 갖고있는 각 샘플의 길이가 500개의 단어로 구성되어있다면 이 값은 500이 됩니다.

Embedding()은 (number of samples, input_length)인 2D 정수 텐서를 입력받습니다. 이 때 각 sample은 정수 인코딩이 된 결과로, 정수의 시퀀스입니다. Embedding()은 워드 임베딩 작업을 수행하고 (number of samples, input_length, embedding word dimentionality)인 3D 실수 텐서를 리턴합니다. 케라스의 임베딩 층(embedding layer)을 사용하는 간단한 실습을 진행해보겠습니다.

2) 임베딩 층 사용하기
RNN 챕터에서 이미 사용한 바 있지만, 임베딩 층을 복습해보겠습니다. 문장의 긍, 부정을 판단하는 감성 분류 모델을 만들어봅시다.

In [2]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np

In [3]:
sentences = ['nice great best amazing', 'stop lies', 'pitiful nerd','excellent work','supreme quality', 'bad','highly respect']
y_train = [1,0,0,1,1,0,1]

In [5]:
t = Tokenizer()
t.fit_on_texts(sentences)

vocab_size = len(t.word_index) + 1

In [8]:
x_encodded = t.texts_to_sequences(sentences)
x_encodded

[[1, 2, 3, 4], [5, 6], [7, 8], [9, 10], [11, 12], [13], [14, 15]]

In [9]:
max_len = max(len(l) for l in x_encodded)
max_len

4

In [10]:
x_train = pad_sequences(x_encodded, maxlen=max_len, padding='post')
y_train = np.array(y_train)

In [11]:
x_train.shape

(7, 4)

In [13]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, Flatten, Input


model = Sequential()
model.add(Embedding(vocab_size, 4, input_length=max_len))
model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))
model.summary()


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 4, 4)              64        
_________________________________________________________________
flatten (Flatten)            (None, 16)                0         
_________________________________________________________________
dense (Dense)                (None, 1)                 17        
Total params: 81
Trainable params: 81
Non-trainable params: 0
_________________________________________________________________


In [14]:
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics='accuracy')
model.fit(x=x_train,
          y=y_train,
          epochs=20,
          batch_size=64)

Epoch 1/20
1/1 [==============================] - 1s 778ms/step - loss: 0.6892 - accuracy: 0.5714
Epoch 2/20
1/1 [==============================] - 0s 8ms/step - loss: 0.6878 - accuracy: 0.5714
Epoch 3/20
1/1 [==============================] - 0s 7ms/step - loss: 0.6864 - accuracy: 0.5714
Epoch 4/20
1/1 [==============================] - 0s 5ms/step - loss: 0.6849 - accuracy: 0.5714
Epoch 5/20
1/1 [==============================] - 0s 6ms/step - loss: 0.6835 - accuracy: 0.8571
Epoch 6/20
1/1 [==============================] - 0s 5ms/step - loss: 0.6820 - accuracy: 0.8571
Epoch 7/20
1/1 [==============================] - 0s 5ms/step - loss: 0.6806 - accuracy: 0.8571
Epoch 8/20
1/1 [==============================] - 0s 7ms/step - loss: 0.6791 - accuracy: 0.8571
Epoch 9/20
1/1 [==============================] - 0s 5ms/step - loss: 0.6777 - accuracy: 0.8571
Epoch 10/20
1/1 [==============================] - 0s 7ms/step - loss: 0.6762 - accuracy: 0.8571
Epoch 11/20
1/1 [====================

In [15]:
model.predict(x_train)

array([[0.53141105],
       [0.49934337],
       [0.48979473],
       [0.52755046],
       [0.50979954],
       [0.48515627],
       [0.5244583 ]], dtype=float32)

In [18]:
import gensim
!wget "https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz"

--2021-05-13 15:28:51--  https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.216.230.205
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.216.230.205|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1647046227 (1.5G) [application/x-gzip]
Saving to: ‘GoogleNews-vectors-negative300.bin.gz’

GoogleNews-vectors- 100%[===================>]   1.53G  67.0MB/s    in 22s     

2021-05-13 15:29:13 (72.2 MB/s) - ‘GoogleNews-vectors-negative300.bin.gz’ saved [1647046227/1647046227]



In [19]:
# 구글의 사전 훈련된 Word2vec 모델을 로드합니다.
word2vec_model = gensim.models.KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin.gz', binary=True)

In [20]:
print(word2vec_model.vectors.shape)

(3000000, 300)


In [21]:
embedding_matrix = np.zeros((vocab_size, 300))
np.shape(embedding_matrix)

(16, 300)

In [22]:
def get_vector(word):
    if word in word2vec_model:
        return word2vec_model[word]
    else:
        return None

In [23]:
for word, i in t.word_index.items(): # 훈련 데이터의 단어 집합에서 단어와 정수 인덱스를 1개씩 꺼내온다.
    print(word, 1 )
    temp = get_vector(word) # 단어(key) 해당되는 임베딩 벡터의 300개의 값(value)를 임시 변수에 저장
    if temp is not None: # 만약 None이 아니라면 임베딩 벡터의 값을 리턴받은 것이므로
        embedding_matrix[i] = temp # 해당 단어 위치의 행에 벡터의 값을 저장한다.

nice 1
great 1
best 1
amazing 1
stop 1
lies 1
pitiful 1
nerd 1
excellent 1
work 1
supreme 1
quality 1
bad 1
highly 1
respect 1


In [24]:
print(embedding_matrix[1])
print('단어 nice의 정수 인덱스 :', t.word_index['nice'])

[ 0.15820312  0.10595703 -0.18945312  0.38671875  0.08349609 -0.26757812
  0.08349609  0.11328125 -0.10400391  0.17871094 -0.12353516 -0.22265625
 -0.01806641 -0.25390625  0.13183594  0.0859375   0.16113281  0.11083984
 -0.11083984 -0.0859375   0.0267334   0.34570312  0.15136719 -0.00415039
  0.10498047  0.04907227 -0.06982422  0.08642578  0.03198242 -0.02844238
 -0.15722656  0.11865234  0.36132812  0.00173187  0.05297852 -0.234375
  0.11767578  0.08642578 -0.01123047  0.25976562  0.28515625 -0.11669922
  0.38476562  0.07275391  0.01147461  0.03466797  0.18164062 -0.03955078
  0.04199219  0.01013184 -0.06054688  0.09765625  0.06689453  0.14648438
 -0.12011719  0.08447266 -0.06152344  0.06347656  0.3046875  -0.35546875
 -0.2890625   0.19628906 -0.33203125 -0.07128906  0.12792969  0.09619141
 -0.12158203 -0.08691406 -0.12890625  0.27734375  0.265625    0.1796875
  0.12695312  0.06298828 -0.34375    -0.05908203  0.0456543   0.171875
  0.08935547  0.14648438 -0.04638672 -0.00842285 -0.0279

In [25]:
model = Sequential()
e = Embedding(vocab_size, 300, weights=[embedding_matrix], input_length=max_len, trainable=False)
model.add(e)
model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='adam', 
              loss='binary_crossentropy',
              metrics='accuracy')

model.fit(x_train, 
          y_train, 
          epochs=100, 
          verbose=2)

Epoch 1/100
1/1 - 0s - loss: 0.7276 - accuracy: 0.4286
Epoch 2/100
1/1 - 0s - loss: 0.7085 - accuracy: 0.5714
Epoch 3/100
1/1 - 0s - loss: 0.6899 - accuracy: 0.5714
Epoch 4/100
1/1 - 0s - loss: 0.6719 - accuracy: 0.7143
Epoch 5/100
1/1 - 0s - loss: 0.6544 - accuracy: 0.7143
Epoch 6/100
1/1 - 0s - loss: 0.6374 - accuracy: 0.7143
Epoch 7/100
1/1 - 0s - loss: 0.6210 - accuracy: 0.8571
Epoch 8/100
1/1 - 0s - loss: 0.6051 - accuracy: 0.8571
Epoch 9/100
1/1 - 0s - loss: 0.5897 - accuracy: 0.8571
Epoch 10/100
1/1 - 0s - loss: 0.5748 - accuracy: 1.0000
Epoch 11/100
1/1 - 0s - loss: 0.5604 - accuracy: 1.0000
Epoch 12/100
1/1 - 0s - loss: 0.5465 - accuracy: 1.0000
Epoch 13/100
1/1 - 0s - loss: 0.5330 - accuracy: 1.0000
Epoch 14/100
1/1 - 0s - loss: 0.5199 - accuracy: 1.0000
Epoch 15/100
1/1 - 0s - loss: 0.5073 - accuracy: 1.0000
Epoch 16/100
1/1 - 0s - loss: 0.4950 - accuracy: 1.0000
Epoch 17/100
1/1 - 0s - loss: 0.4832 - accuracy: 1.0000
Epoch 18/100
1/1 - 0s - loss: 0.4717 - accuracy: 1.0000
E